In [42]:
import numpy as np

In [6]:
import logging
import time
import pandas as pd
from colorama import Fore

from deeppavlov import build_model, configs


logging.basicConfig(level=logging.ERROR)


def timed_inference(model, question: str, context: str, print_q_v: bool = True):
    start_time = time.perf_counter()
    prediction = model([context], [question])[0][0]
    predict_time = time.perf_counter() - start_time

    if print_q_v:
        print(Fore.BLUE + f"Question: {question}")
        print(Fore.YELLOW + f"Variants: {context}")
    print(Fore.GREEN + f"[{predict_time:.3f}] Predicted: `{prediction}`")


In [88]:
model = build_model(configs.squad.squad_ru_rubert, download=True)

/Users/ruagmn9/Library/Caches/pypoetry/virtualenvs/deeppavlov-test-8ztkSxjl-py3.7/lib/python3.7/site-packages/cryptography/hazmat/backends/openssl/x509.py:18: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
2021-11-12 15:08:23.256 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/deeppavlov_data/squad_model_ru_rubert.tar.gz to /Users/ruagmn9/.deeppavlov/squad_model_ru_rubert.tar.gz
INFO:deeppavlov.core.data.utils:Downloading from http://files.deeppavlov.ai/deeppavlov_data/squad_model_ru_rubert.tar.gz to /Users/ruagmn9/.deeppavlov/squad_model_ru_rubert.tar.gz
100%|██████████| 659M/659M [00:39<00:00, 16.8MB/s] 
2021-11-12 15:09:02.888 INFO in 'deeppavlov.core.data.utils'['utils'] at line 268: Extracting /Users/ruagmn9/.deeppavlov/squad_model_ru_rubert.tar.gz archive into /Users/ruagmn9/.deeppavlov/models
INFO:deeppavlov.core.data.ut

ModuleNotFoundError: No module named 'bert_dp'

In [102]:

model([ "Поплавать. Погулять. Покурить. Попрыгать."], ["Раз два три четыре пять вышел зайчик что делать?"])


[['Погулять'], [11], [857.1954345703125]]

In [90]:
data = pd.read_csv('/Users/ruagmn9/PycharmProjects/raif-bootcamp-2021/data/boot_camp_train.csv')

In [91]:
data = data.rename(columns = {'Вопрос': 'question', 'Правильный ответ': 'target'})

In [92]:
data = data.dropna(subset=['question']).fillna('Пусто')

In [93]:
data['context'] = data[['1', '2', '3', '4']].agg('. '.join, axis=1)

In [94]:
%%time

data['predict'] = data.apply(lambda x: model([x['context']], [x['question']])[0][0], axis=1)

CPU times: user 6min 51s, sys: 2min 5s, total: 8min 56s
Wall time: 1min 43s


In [103]:
data['rank'] = data.apply(lambda x: model([x['context']], [x['question']])[1][0], axis=1)

In [104]:
data['score'] = data.apply(lambda x: model([x['context']], [x['question']])[2][0], axis=1)

In [108]:
data.head()

,Unnamed: 0,question,1,2,3,4,target,context,predict,predict_index,rank,score
0,0,"Как же, согласно поговорке, все-таки там, где ...",Тепло,Сытно,Хорошо,Весело,3.0,Тепло. Сытно. Хорошо. Весело,Хорошо,3,14,1100.953613
1,4,Какую спортивную игру начинают не с подачи?,Теннис,Волейбол,Баскетбол,Бадминтон,3.0,Теннис. Волейбол. Баскетбол. Бадминтон,Волейбол,2,8,122.065384
2,9,На чьи стихи написана песня «День победы?»,Добронравов,Матусовский,Харитонов,Ошанин,3.0,Добронравов. Матусовский. Харитонов. Ошанин,Матусовский,2,13,591.961975
3,10,Кто регулярно тушит свет на канале НТВ?,Лев Новожёнов,Леонид Парфёнов,Елена Ханга,Анатолий Чубайс,1.0,Лев Новожёнов. Леонид Парфёнов. Елена Ханга. А...,Лев,1,0,270.551361
4,11,"По мнению пословицы, плох солдат, который не м...",Маршалом,Генералом,Полковником,Дембелем,2.0,Маршалом. Генералом. Полковником. Дембелем,Дембелем,4,34,529.799255


In [118]:
def get_score(df):
    return (df['target'] == df['predict_index']).sum() / df.shape[0]

In [124]:
get_score(data[data['score'] < 10])

0.3333333333333333

In [112]:
data[data['rank'] < 10].head()

,Unnamed: 0,question,1,2,3,4,target,context,predict,predict_index,rank,score
1,4,Какую спортивную игру начинают не с подачи?,Теннис,Волейбол,Баскетбол,Бадминтон,3.0,Теннис. Волейбол. Баскетбол. Бадминтон,Волейбол,2,8,122.065384
3,10,Кто регулярно тушит свет на канале НТВ?,Лев Новожёнов,Леонид Парфёнов,Елена Ханга,Анатолий Чубайс,1.0,Лев Новожёнов. Леонид Парфёнов. Елена Ханга. А...,Лев,1,0,270.551361
5,13,В основе сюжета какого телесериала не лежат кр...,«Убойная сила»,«Улицы разбитых фонарей»,«Петербургские тайны»,«Бандитский Петербург»,3.0,«Убойная сила». «Улицы разбитых фонарей». «Пет...,Убойная сила». «Улицы разбитых фонарей». «Пете...,2,1,222.603073
6,16,"У какого народа утвердительный, в нашем понима...",Болгары,Чехи,Венгры,Поляки,1.0,Болгары. Чехи. Венгры. Поляки,Чехи,2,9,1295.444092
7,18,Какой фильм Сергея Герасимова стал дебютным дл...,«Молодая гвардия»,«Семеро смелых»,«Юность Петра»,«Тихий Дон»,1.0,«Молодая гвардия». «Семеро смелых». «Юность Пе...,Молодая,1,1,136.600723


In [106]:
data.head()

,Unnamed: 0,question,1,2,3,4,target,context,predict,predict_index,rank,score
0,0,"Как же, согласно поговорке, все-таки там, где ...",Тепло,Сытно,Хорошо,Весело,3.0,Тепло. Сытно. Хорошо. Весело,Хорошо,3,14,1100.953613
1,4,Какую спортивную игру начинают не с подачи?,Теннис,Волейбол,Баскетбол,Бадминтон,3.0,Теннис. Волейбол. Баскетбол. Бадминтон,Волейбол,2,8,122.065384
2,9,На чьи стихи написана песня «День победы?»,Добронравов,Матусовский,Харитонов,Ошанин,3.0,Добронравов. Матусовский. Харитонов. Ошанин,Матусовский,2,13,591.961975
3,10,Кто регулярно тушит свет на канале НТВ?,Лев Новожёнов,Леонид Парфёнов,Елена Ханга,Анатолий Чубайс,1.0,Лев Новожёнов. Леонид Парфёнов. Елена Ханга. А...,Лев,1,0,270.551361
4,11,"По мнению пословицы, плох солдат, который не м...",Маршалом,Генералом,Полковником,Дембелем,2.0,Маршалом. Генералом. Полковником. Дембелем,Дембелем,4,34,529.799255


In [96]:
def select_answer_based_on_predict(predict_str,
                                  series):
    for index, answer in enumerate(series, 1 ):
        if predict_str in answer:
            return index
    return 2

In [97]:
data['predict_index'] = data.apply(lambda x: select_answer_based_on_predict(x['predict'],
                                                                            x[['1','2', '3', '4']]),
                                                                            axis=1)

In [98]:
data

,Unnamed: 0,question,1,2,3,4,target,context,predict,predict_index
0,0,"Как же, согласно поговорке, все-таки там, где ...",Тепло,Сытно,Хорошо,Весело,3.0,Тепло. Сытно. Хорошо. Весело,Хорошо,3
1,4,Какую спортивную игру начинают не с подачи?,Теннис,Волейбол,Баскетбол,Бадминтон,3.0,Теннис. Волейбол. Баскетбол. Бадминтон,Волейбол,2
2,9,На чьи стихи написана песня «День победы?»,Добронравов,Матусовский,Харитонов,Ошанин,3.0,Добронравов. Матусовский. Харитонов. Ошанин,Матусовский,2
3,10,Кто регулярно тушит свет на канале НТВ?,Лев Новожёнов,Леонид Парфёнов,Елена Ханга,Анатолий Чубайс,1.0,Лев Новожёнов. Леонид Парфёнов. Елена Ханга. А...,Лев,1
4,11,"По мнению пословицы, плох солдат, который не м...",Маршалом,Генералом,Полковником,Дембелем,2.0,Маршалом. Генералом. Полковником. Дембелем,Дембелем,4
...,...,...,...,...,...,...,...,...,...,...
8995,9266,Как называется хищное растение?,«Венерина мухоловка»,«Марсова пчелоедка»,«Сатурнова клоподавка»,«Юпитеров слепнежуй»,1.0,«Венерина мухоловка». «Марсова пчелоедка». «Са...,мухоловка,1
8996,9267,Что бывает у шапки?,«Полянка»,«Тропинка»,«Опушка»,«Чащоба»,3.0,«Полянка». «Тропинка». «Опушка». «Чащоба»,Полянка». «Тропинка,2
8997,9268,Что не использовалось для разогрева утюгов раз...,Скипидар,Уголь,Спирт,Электричество,1.0,Скипидар. Уголь. Спирт. Электричество,Скипидар,1
8998,9269,В каком турнире капитан команды обычно не выхо...,Кубок Дэвиса по теннису,Кубок Стэнли по хоккею,Кубок Европы по баскетболу,Кубок мира по борьбе,1.0,Кубок Дэвиса по теннису. Кубок Стэнли по хокке...,Дэвиса,1


In [99]:
from sklearn.metrics import accuracy_score

In [100]:
accuracy_score(data['target'], data['predict_index'])

0.25769529947771974